In [2]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

## Primer Cuatrimestre de 2019. Examen parcial, primera oportunidad

In [5]:
df1 = spark.read.csv("bases.csv",header=False,sep=";");
print(df1.collect())

AnalysisException: u'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'

In [ ]:
df2 = spark.read.csv("mediciones.csv",header=False,sep=";");
print(df2.collect())

Posible solución.

a. Map por ID_BASE en ambos RDD.

b.Filter en RDD bases por PCIA = Buenos Aires.

c.Filter en RDD mediciones por TIMESTAMP entre 01-12-2017 y 31-12-2018.

d.Inner join entre ambos RDD.

e.MAP por ID_BASE, YEAR(TIMESTAMP), MONTH(TIMESTAMP).

f.ReduceByKey para calcular suma y contador.

g.MAP para cambiar la clave a ID_BASE y aprovechar para hacer el calculo del promedio. En los values debe
quedar el promedio, el nombre de la base y el mes/año.

h.GroupByKey

i.Filter para comparar que la tasa de un mes a otro haya variado en 50%.

j.Map para quedarse con ID_BASE y nombre.

Map innecesarios, descuento de 3 puntos. Filter tardíos

In [ ]:
bases = df1.rdd
mediciones = df2.rdd

In [ ]:
bases.collect()

In [ ]:
mediciones.collect()

In [ ]:
bases = bases.filter(lambda x: x[2] == 'BUENOSAIRES')

In [ ]:
bases.collect()

In [ ]:
mediciones = mediciones.filter(lambda x: x[0].endswith('2018'))

In [ ]:
mediciones.collect()

In [ ]:
bases = bases.map(lambda x: (x[0],(x[1],x[3])))

In [ ]:
bases.collect()

In [ ]:
mediciones = mediciones.map(lambda x: (x[1],(x[2],x[0][3:5],x[0][6:10])))

In [ ]:
mediciones.collect()

In [ ]:
joins = bases.leftOuterJoin(mediciones)

In [ ]:
joins.collect()

In [ ]:
joins = joins.map(lambda x: ((x[0],x[1][1][1],x[1][1][2]),(x[1][1][0],1,x[1][0][0])))

In [ ]:
joins.collect()

In [ ]:
joins = joins.reduceByKey(lambda x, y: (int(x[0])+int(y[0]),x[1]+y[1],x[2]))

In [ ]:
joins.collect()

In [ ]:
joins = joins.map(lambda x: (x[0][0],(x[1][0]/x[1][1], x[1][2], x[0][1],x[0][2])))

In [ ]:
joins.collect()

In [ ]:
group = joins.groupByKey()

In [ ]:
group.collect()

In [ ]:
def filter_10p(x):
    data = list(x[1])
    print(data)
    if int(data[1][2]) == 12:
        return (abs(int(data[1][0]) - int(data[2][0])) >= 0.5*int(data[2][0]))
    else: return False

In [ ]:
resultado = group.filter(filter_10p).map(lambda x: x[0], list(x[1])[0][1])

In [ ]:
resultado.collect()